In [42]:
import pandas as pd
import numpy as np
import re
from tqdm.notebook import tqdm
from scipy.spatial.transform import Rotation
import glob
import os
import json

In [43]:
p_space = re.compile('( *)(.*)')

def remove_space(x):
    name = p_space.match(x)
    return name.group(2)

# extracting transformation parameters (rotation angles and translation vector) from csv
def extract_transformation(x):
    # converting to rotations
    r = Rotation.from_euler('XYZ', [x['pose_Rx'], x['pose_Ry'], x['pose_Rz']], degrees=False)
    # converting to translation vector
    translation = np.array([x['pose_Tx'], x['pose_Ty'], x['pose_Tz']])
    return r, translation


# mapping global coordinate to local ones using the inverse transformation defined by the Pose parameters
def map_row_to_local_quat(x):
    r, translation = extract_transformation(x)
    # mapping each of the 68 landmarks points to local coordinates
    for i in range(0, 68):
        # converting to location vector
        vec_global = np.array([x['X_' + str(i)], x['Y_' + str(i)], x['Z_' + str(i)]])
        # translating to object space and inverting rotation
        vec_local = r.apply((vec_global - translation), inverse=True)
        # updating row with local coordinates
        x['X_L_' + str(i)] = vec_local[0]
        x['Y_L_' + str(i)] = vec_local[1]
        x['Z_L_' + str(i)] = vec_local[2]
    # Each row is a (possibly non-unit norm) quaternion in scalar-last (x, y, z, w) format.
    quat = r.as_quat()
    x['quat_x'] = quat[0]  # x
    x['quat_y'] = quat[1]  # y
    x['quat_z'] = quat[2]  # z
    x['quat_w'] = quat[3]  # w
    return x

In [44]:
# please add an absolute path to the processed directory of OpenFace
video_path = "C:\\Users\\morda\\Downloads\\OpenFace_2.2.0_win_x64\\to_processed_2\\"
paths = glob.glob(video_path + "*.csv")

# please specify a location where to save the processed CSVs
save_location = "C:\\Users\\morda\\Downloads\\OpenFace_2.2.0_win_x64\\after_processed" 

In [45]:
# preprocessing
for path in tqdm(paths):
    df = pd.read_csv(path)
    df = df.rename(columns=remove_space)
    df = df.apply(map_row_to_local_quat, axis=1)
    df = df.drop(['face_id', 'success', 'timestamp', 'frame','confidence',
                  'pose_Tz', 'pose_Ty', 'pose_Tx', 'pose_Rx', 'pose_Ry',
                  'pose_Rz'], axis=1)
    for i in range(0, 68):
        df = df.drop(["X_" + str(i), "Y_" + str(i), "Z_" + str(i)], axis=1)
    base_name = os.path.basename(path)
    df.to_csv(str(save_location) + "\\" + str(base_name) + "_proc" + ".csv")
    df.to_json(str(save_location) + "\\" + str(base_name) + "_proc" + ".json", orient="index")

In [38]:
# an example of how the CSV looks
df.head()

X_0   X_1   X_2   X_3   X_4   X_5   X_6  X_7   X_8   X_9  ...    X_L_66  \
0 -75.8 -74.9 -71.7 -66.1 -55.4 -39.3 -21.0 -1.1  16.9  33.0  ...  2.532862   
1 -74.7 -74.1 -71.4 -66.7 -57.0 -41.9 -24.4 -4.7  13.5  30.2  ...  2.470742   
2 -74.2 -73.9 -71.5 -67.2 -57.8 -42.9 -25.5 -5.8  12.3  28.8  ...  2.344361   
3 -74.8 -74.6 -72.1 -67.8 -58.3 -43.5 -26.3 -7.0  10.9  27.5  ...  2.130660   
4 -74.3 -74.2 -71.9 -67.9 -58.9 -44.7 -27.9 -8.8   9.2  25.9  ...  2.039595   

      Y_L_66     Z_L_66    X_L_67     Y_L_67     Z_L_67    quat_x    quat_y  \
0  28.365933 -11.532061 -4.260249  28.015736 -11.156558 -0.074180 -0.058131   
1  28.573331 -11.216069 -4.491214  28.168396 -10.601909 -0.064809 -0.052833   
2  28.787225 -10.930676 -4.707377  28.235835 -10.283395 -0.064673 -0.049011   
3  28.823722 -10.868917 -4.719309  28.418829 -10.111295 -0.060547 -0.042660   
4  28.907455 -10.601373 -4.803917  28.446437  -9.727414 -0.062344 -0.037390   

     quat_z    quat_w  
0 -0.033088  0.994999  
1 -0.026504  0.996146  
2 -0.021784  0.996464  
3 -0.017893  0.997093  
4 -0.013161  0.997267  

[5 rows x 412 columns]

In [16]:
# an exmaple of how the json looks like
with open(str(save_location) + "\\" + str(base_name) + "_proc" + ".json", encoding='utf-8') as data_file:
    data = json.loads(data_file.read())
data

{'0': {'X_0': -80.4,
  'X_1': -81.8,
  'X_2': -80.1,
  'X_3': -76.5,
  'X_4': -70.3,
  'X_5': -59.8,
  'X_6': -47.4,
  'X_7': -32.1,
  'X_8': -15.6,
  'X_9': 1.0,
  'X_10': 18.1,
  'X_11': 35.5,
  'X_12': 50.3,
  'X_13': 60.4,
  'X_14': 65.7,
  'X_15': 68.1,
  'X_16': 70.2,
  'X_17': -70.6,
  'X_18': -61.7,
  'X_19': -50.0,
  'X_20': -38.2,
  'X_21': -27.7,
  'X_22': -1.1,
  'X_23': 11.2,
  'X_24': 23.5,
  'X_25': 36.0,
  'X_26': 46.7,
  'X_27': -14.9,
  'X_28': -15.7,
  'X_29': -16.6,
  'X_30': -17.6,
  'X_31': -28.3,
  'X_32': -22.3,
  'X_33': -16.2,
  'X_34': -8.9,
  'X_35': -2.0,
  'X_36': -56.8,
  'X_37': -48.8,
  'X_38': -39.0,
  'X_39': -30.6,
  'X_40': -39.2,
  'X_41': -48.9,
  'X_42': 6.1,
  'X_43': 14.1,
  'X_44': 23.3,
  'X_45': 31.2,
  'X_46': 23.9,
  'X_47': 14.8,
  'X_48': -39.6,
  'X_49': -30.5,
  'X_50': -21.7,
  'X_51': -15.7,
  'X_52': -9.1,
  'X_53': 1.3,
  'X_54': 12.9,
  'X_55': 1.5,
  'X_56': -8.6,
  'X_57': -15.8,
  'X_58': -22.3,
  'X_59': -30.9,
  'X_60': -35.0

In [39]:
for i in range(0, 68):
    print (["X_" + str(i), "Y_" + str(i), "Z_" + str(i)])

['X_0', 'Y_0', 'Z_0']
['X_1', 'Y_1', 'Z_1']
['X_2', 'Y_2', 'Z_2']
['X_3', 'Y_3', 'Z_3']
['X_4', 'Y_4', 'Z_4']
['X_5', 'Y_5', 'Z_5']
['X_6', 'Y_6', 'Z_6']
['X_7', 'Y_7', 'Z_7']
['X_8', 'Y_8', 'Z_8']
['X_9', 'Y_9', 'Z_9']
['X_10', 'Y_10', 'Z_10']
['X_11', 'Y_11', 'Z_11']
['X_12', 'Y_12', 'Z_12']
['X_13', 'Y_13', 'Z_13']
['X_14', 'Y_14', 'Z_14']
['X_15', 'Y_15', 'Z_15']
['X_16', 'Y_16', 'Z_16']
['X_17', 'Y_17', 'Z_17']
['X_18', 'Y_18', 'Z_18']
['X_19', 'Y_19', 'Z_19']
['X_20', 'Y_20', 'Z_20']
['X_21', 'Y_21', 'Z_21']
['X_22', 'Y_22', 'Z_22']
['X_23', 'Y_23', 'Z_23']
['X_24', 'Y_24', 'Z_24']
['X_25', 'Y_25', 'Z_25']
['X_26', 'Y_26', 'Z_26']
['X_27', 'Y_27', 'Z_27']
['X_28', 'Y_28', 'Z_28']
['X_29', 'Y_29', 'Z_29']
['X_30', 'Y_30', 'Z_30']
['X_31', 'Y_31', 'Z_31']
['X_32', 'Y_32', 'Z_32']
['X_33', 'Y_33', 'Z_33']
['X_34', 'Y_34', 'Z_34']
['X_35', 'Y_35', 'Z_35']
['X_36', 'Y_36', 'Z_36']
['X_37', 'Y_37', 'Z_37']
['X_38', 'Y_38', 'Z_38']
['X_39', 'Y_39', 'Z_39']
['X_40', 'Y_40', 'Z_40']
['X_4